In [1]:
# Surprise 설치
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 4.8 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619416 sha256=b831344f7424bd437235f6b43afd62d32cc6d29dc51a5877e0dd78a0e2b78cc1
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor
from surprise import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import BaselineOnly, CoClustering
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

In [72]:
ratings=pd.read_csv("./drive/MyDrive/data-files/ratings1.csv")
print(ratings.shape)
ratings.head(1)

(199999, 4)


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044


In [73]:
# null 체크
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [74]:
ratings.groupby('userId')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

,userId,rating
547,548,3212
846,847,2920
996,997,2371
625,626,1888
1400,1401,1531
186,187,1387
756,757,1355
1202,1203,1311
802,803,1286
1227,1228,1258


In [75]:
ratings.groupby('movieId')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

,movieId,rating
333,356,712
297,318,692
276,296,692
546,593,640
2189,2571,624
243,260,582
448,480,553
493,527,518
104,110,507
543,589,496


In [76]:
min_movie_ratings = 50
filter_movie = ratings['movieId'].value_counts() > min_movie_ratings
filter_movie = filter_movie[filter_movie].index.tolist()

min_user_ratings = 50
filter_users = ratings['userId'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

ratings_new = ratings[(ratings['movieId'].isin(filter_movie)) & (ratings['userId'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(ratings.shape))
print('The new data frame shape:\t{}'.format(ratings_new.shape))

The original data frame shape:	(199999, 4)
The new data frame shape:	(108855, 4)


In [77]:
reader = Reader(rating_scale=(0, 9))
data = Dataset.load_from_df(df_new[['userId', 'movieId', 'rating']], reader)

In [78]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), 
                  KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.823890,145.572112,7.614487
KNNBaseline,0.830738,0.540316,7.679633
SVD,0.838227,4.398383,0.425381
KNNWithZScore,0.841579,0.375970,7.135072
SlopeOne,0.841727,0.603867,5.329138
BaselineOnly,0.841840,0.279668,0.339213
KNNWithMeans,0.842457,0.318603,6.639901
NMF,0.859679,4.947541,0.355205
CoClustering,0.869196,2.107802,0.297659


In [42]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = KNNBaseline(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Using ALS
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


{'fit_time': (0.37879467010498047, 0.41136646270751953, 0.43912553787231445),
 'test_rmse': array([0.82835931, 0.8326787 , 0.83205125]),
 'test_time': (7.6080193519592285, 7.787092685699463, 7.560332775115967)}

In [47]:
trainset, testset = train_test_split(data, test_size=0.25)
algo = KNNBaseline(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8241


0.8240680314544292

In [52]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
ratings = pd.DataFrame(predictions, columns=['userId', 'movieId', 'rating', 'est', 'details'])
ratings['Iu'] = ratings.userId.apply(get_Iu)
ratings['Ui'] = ratings.movieId.apply(get_Ui)
ratings['err'] = abs(ratings.est - ratings.rating)
best_predictions = ratings.sort_values(by='err')[:10]
worst_predictions = ratings.sort_values(by='err')[-10:]

In [53]:
best_predictions

,userId,movieId,rating,est,details,Iu,Ui,err
13127,1242,4018,3.5,3.500025,"{'actual_k': 40, 'was_impossible': False}",247,61,0.000025
13250,94,32,4.0,3.999970,"{'actual_k': 40, 'was_impossible': False}",39,243,0.000030
11030,1253,6874,4.0,4.000062,"{'actual_k': 40, 'was_impossible': False}",52,186,0.000062
17193,1367,4571,3.0,3.000157,"{'actual_k': 40, 'was_impossible': False}",302,44,0.000157
14203,824,1222,4.0,3.999832,"{'actual_k': 40, 'was_impossible': False}",40,173,0.000168
23601,12,33679,3.0,2.999807,"{'actual_k': 40, 'was_impossible': False}",286,65,0.000193
12754,1040,84152,4.0,4.000246,"{'actual_k': 40, 'was_impossible': False}",101,54,0.000246
12184,901,8961,4.0,3.999656,"{'actual_k': 40, 'was_impossible': False}",329,170,0.000344
13087,318,1258,4.0,4.000409,"{'actual_k': 40, 'was_impossible': False}",211,148,0.000409
13819,741,2355,4.0,3.999588,"{'actual_k': 40, 'was_impossible': False}",161,137,0.000412


In [54]:
worst_predictions

,userId,movieId,rating,est,details,Iu,Ui,err
12658,712,17,0.5,4.130209,"{'actual_k': 40, 'was_impossible': False}",91,78,3.630209
14838,174,5952,0.5,4.150599,"{'actual_k': 40, 'was_impossible': False}",143,242,3.650599
9853,402,46578,0.5,4.154599,"{'actual_k': 40, 'was_impossible': False}",130,101,3.654599
19039,368,112556,0.5,4.175602,"{'actual_k': 40, 'was_impossible': False}",129,67,3.675602
24634,756,1207,0.5,4.190973,"{'actual_k': 40, 'was_impossible': False}",219,81,3.690973
5599,1296,2064,0.5,4.203218,"{'actual_k': 35, 'was_impossible': False}",55,35,3.703218
20736,715,1953,0.5,4.213058,"{'actual_k': 40, 'was_impossible': False}",24,46,3.713058
26764,756,912,0.5,4.273617,"{'actual_k': 40, 'was_impossible': False}",219,134,3.773617
16952,399,4973,0.5,4.295267,"{'actual_k': 40, 'was_impossible': False}",54,183,3.795267
3624,240,750,1.0,5.519212,"{'actual_k': 40, 'was_impossible': False}",97,140,4.519212
